Задание 1.

Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>

In [436]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [437]:
KEYWORDS = ['python', 'парсинг']

In [443]:
def find_in_harb_posts(keys):
    '''функция собирает новости по ключевым словам с ресурса harb.ru'''
    '''на вход подается список ключевых слов'''
    '''на выходе формируется датафрейм с иформацией'''
    news = pd.DataFrame()
    req = requests.get('https://habr.com/ru/all')
    soup = BeautifulSoup(req.text, 'html.parser')
    posts = soup.find_all('article', class_='post')
    news = pd.DataFrame()
    for post in posts:
        next_link = post.find('a', class_='post__title_link').get('href')
        ref = requests.get(next_link)
        ref_soup = BeautifulSoup(ref.text, 'html.parser')
        date = pd.to_datetime(ref_soup.find('span', class_ = 'post__time').get('data-time_published'), format='%Y%m%d %H:%M') 
        title = ref_soup.title.text
        body = ref_soup.find('div', class_ ='post__text').text.strip()
        for key in keys:
            if key in title.lower() or key in body.lower():
                bloc = {'дата': date, 'заголовок': title, 'ссылка': next_link, 'текст': body}
                news = pd.concat([news, pd.DataFrame([bloc])])
    return news

In [444]:
find_in_harb_posts(KEYWORDS)

,дата,заголовок,ссылка,текст
0,2020-11-01 15:36:00+00:00,Из ничего к ЦОД с VXLAN/EVPN или как готовить ...,https://habr.com/ru/post/526022/,В последние полгода удалось поработать над бол...
0,2020-11-01 13:30:00+00:00,FOSS News №40 – дайджест новостей и других мат...,https://habr.com/ru/post/526014/,Всем привет!\n\r\nПродолжаем дайджесты новосте...


Задание 2.
Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck https://www.avast.com/hackcheck/. 
Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

Дополнительная часть (необязательная)
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.getGROUP = 'netology'
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.


In [165]:
import json
import requests

In [424]:
def email_leak_check(email):
    '''Функция проверяет емэйл на утечку в сети через сервис Avast'''
    '''на вход подается емэйл или список из емэйлов'''
    '''на выходе формируется датафрейм с информацией об утечке'''
    '''предварительно необходимо подгрузить библиотеки json и requests'''
    if type(email) == list:
        email_list = []
        site_list = []
        descript_list = []
        data_list = []
        for mail in email:    
            json = { 'emailAddresses' :[mail]}
            headers = {'Vaar-Header-App-Product': 'hackcheck-web-avast','Vaar-Version': '0'}
            url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
            avast = requests.post(url=url, json=json, headers=headers)
            for key, value in avast.json().items():
                for data, data_info in value.items():
                    for title, info in data_info.items():
                        if title == 'site':
                            email_list.append(list(avast.json()['summary'].keys())[0])
                            site_list.append(info)
                        elif title == 'description':
                            descript_list.append(info)
                        elif title == 'publishDate':
                            data_list.append(info) 
        leak_df = pd.DataFrame({'email': email_list, 'date': data_list, 'site': site_list, 'description': descript_list})
        return leak_df
    else:          
        json = { 'emailAddresses' :[email]}
        headers = {'Vaar-Header-App-Product': 'hackcheck-web-avast','Vaar-Version': '0'}
        url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
        avast = requests.post(url=url, json=json, headers=headers)
        email_list = []
        site_list = []
        descript_list = []
        data_list = []
        for key, value in avast.json().items():
                for data, data_info in value.items():
                    for title, info in data_info.items():
                        if title == 'site':
                            email_list.append(list(avast.json()['summary'].keys())[0])
                            site_list.append(info)
                        elif title == 'description':
                            descript_list.append(info)
                        elif title == 'publishDate':
                            data_list.append(info) 
        leak_df = pd.DataFrame({'email': email_list, 'date': data_list, 'site': site_list, 'description': descript_list})  
        return leak_df

In [425]:
EMAIL = ['xxx@x.ru', 'yyy@y.com']

In [426]:
email_leak_check(EMAIL)

,email,date,site,description
0,xxx@x.ru,2017-02-14T00:00:00Z,parapa.mail.ru,"In July and August 2016, two criminals execute..."
1,xxx@x.ru,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
2,xxx@x.ru,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
3,xxx@x.ru,2017-02-14T00:00:00Z,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
4,xxx@x.ru,2017-01-31T00:00:00Z,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
5,xxx@x.ru,2016-10-23T00:00:00Z,imesh.com,"In June 2016, a cache of over 51 million user ..."
6,yyy@y.com,2020-01-03T00:00:00Z,azcentral.com,"At an unconfirmed date, online Arizona newspap..."
7,yyy@y.com,2020-05-28T00:00:00Z,wishbone.io,"In January 2020, the online poll website Wishb..."
8,yyy@y.com,2017-11-04T00:00:00Z,myheritage.com,"In October 2017, a customer database belonging..."
9,yyy@y.com,2019-06-13T00:00:00Z,canva.com,"In May 2019, graphic-design site Canva's datab..."
